In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Read data
data = pd.read_csv('../input/datasets-for-churn-telecom/cell2celltrain.csv')
data = data.fillna(0)

In [ ]:
categorical_data = data.select_dtypes(include=['object', 'category'])
categorical_cols = list(categorical_data.columns)
binary_cols = []

for col in categorical_cols:
    if data[col].nunique() == 2:
        binary_cols.append(col)

for col in binary_cols:
    categorical_cols.remove(col)

numerical_cols = list(data.drop(categorical_data.columns, axis=1).columns)

binary_cols.remove('Churn')
print('binary columns: ', binary_cols)

categorical_cols.remove('ServiceArea')
print('categorical columns: ',categorical_cols)

numerical_cols.remove('CustomerID')
print('numerical columns: ',numerical_cols)

In [ ]:
# Remove churn label from data and store it into xtrain variable
xtrain = data.drop(['Churn', 'CustomerID', 'ServiceArea'], axis=1)
ytrain = data['Churn']

xtr_binary = xtrain[binary_cols]
xtr_cat = xtrain[categorical_cols]
xtr_numerical = xtrain[numerical_cols]

# pd.options.display.max_columns = None
# pd.options.display.max_rows = None

In [ ]:
labelencoder = LabelEncoder()
for col in binary_cols:
    xtr_binary[col] = pd.DataFrame(data=labelencoder.fit_transform(xtr_binary[col]), columns=[col])

xtr_binary.head(10)

In [ ]:
one_hot_df = pd.get_dummies(xtr_cat)
one_hot_df.head()

In [ ]:
numerical_df = StandardScaler().fit_transform(xtr_numerical)
numerical_df = pd.DataFrame(data=numerical_df, columns = numerical_cols)
numerical_df.isnull().sum()

In [ ]:
xtrain_final = pd.concat([numerical_df, xtr_binary, one_hot_df], axis=1)
xtrain_final.head()

In [ ]:
# PCA Projection
pca = PCA(n_components=.90)
principalComponents = pca.fit_transform(xtrain_final.values)
print(principalComponents.shape)
PCDF = pd.DataFrame(data = principalComponents)
PCDF.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(PCDF, ytrain, test_size=0.3,random_state=109)

In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
# svclassifier = SVC(kernel='rbf')
# svclassifier = SVC(kernel='sigmoid')
svclassifier.fit(X_train, y_train)

In [ ]:
y_pred = svclassifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))